<a href="https://colab.research.google.com/github/Mindauskas/bbc_news/blob/master/bbc_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
   %tensorflow_version 2.x
except Exception:
   pass

TensorFlow 2.x selected.


In [0]:
SETUP = True

In [3]:
if SETUP:
    !pip install -q -U toai==0.2.*
    !pip install -q -U nb_black
    !pip install -q -U tensorflow-datasets
    !pip install -q -U --no-deps tensorflow-addons
    !pip install -q -U tensorflow_hub
    !pip install -q -U git+https://github.com/huggingface/transformers
    print(__import__("toai").__version__)
    print(__import__("tensorflow").__version__)

0.2.7
2.0.0


In [4]:
from toai.imports import *
from toai.data import DataBundle, DataParams, DataContainer
from toai.metrics import sparse_top_2_categorical_accuracy
from toai.models import save_keras_model, load_keras_model
from toai.utils import save_file, load_file
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import csv
import transformers

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/toai/imports.py:70: UserWarning: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
  warnings.warn(str(error))


In [5]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
DATA_DIR = Path("data/bbc_news")
DATA_DIR.mkdir(parents=True, exist_ok=True)

TEMP_DIR = Path("temp/bbc_news")
TEMP_DIR.mkdir(parents=True, exist_ok=True)

In [0]:
if SETUP:
    shutil.rmtree(str(DATA_DIR))
    shutil.rmtree(str(TEMP_DIR))
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    TEMP_DIR.mkdir(parents=True, exist_ok=True)
    #shutil.copy("bbc_news.pickle", DATA_DIR)

In [0]:
BATCH_SIZE = 8
AUTOTUNE = tf.data.experimental.AUTOTUNE

# **Dataset**

The datset is taken from https://www.kaggle.com/pariza/bbc-news-summary.
This dataset for extractive text summarization has four hundred and seventeen political news articles of BBC from 2004 to 2005 in the News Articles folder. For each articles, five summaries are provided in the Summaries folder. The first clause of the text of articles is the respective title.

In [0]:
with open("bbc_news.pickle", "rb") as f:
    data = pickle.load(f)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
category             200853 non-null object
headline             200853 non-null object
authors              200853 non-null object
link                 200853 non-null object
short_description    200853 non-null object
date                 200853 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


In [11]:
data.describe(include="all")

,category,headline,authors,link,short_description,date
count,200853,200853,200853,200853,200853,200853
unique,41,199344,27993,200812,178353,2309
top,POLITICS,Sunday Roundup,,https://www.huffingtonpost.comhttps://www.wash...,,2013-01-17 00:00:00
freq,32739,90,36620,2,19712,100
first,NaN,NaN,NaN,NaN,NaN,2012-01-28 00:00:00
last,NaN,NaN,NaN,NaN,NaN,2018-05-26 00:00:00


In [12]:
data.head(5).T

,0,1,2,3,4
category,CRIME,ENTERTAINMENT,ENTERTAINMENT,ENTERTAINMENT,ENTERTAINMENT
headline,There Were 2 Mass Shootings In Texas Last Week...,Will Smith Joins Diplo And Nicky Jam For The 2...,Hugh Grant Marries For The First Time At Age 57,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Julianna Margulies Uses Donald Trump Poop Bags...
authors,Melissa Jeltsen,Andy McDonald,Ron Dicker,Ron Dicker,Ron Dicker
link,https://www.huffingtonpost.com/entry/texas-ama...,https://www.huffingtonpost.com/entry/will-smit...,https://www.huffingtonpost.com/entry/hugh-gran...,https://www.huffingtonpost.com/entry/jim-carre...,https://www.huffingtonpost.com/entry/julianna-...
short_description,She left her husband. He killed their children...,Of course it has a song.,The actor and his longtime girlfriend Anna Ebe...,The actor gives Dems an ass-kicking for not fi...,"The ""Dietland"" actress said using the bags is ..."
date,2018-05-26 00:00:00,2018-05-26 00:00:00,2018-05-26 00:00:00,2018-05-26 00:00:00,2018-05-26 00:00:00


In [0]:
def merge_str(row):
  return row['headline'] + '. ' + row['short_description']

data["merged_text"] = data.apply(merge_str, axis=1)
data = data[['category','merged_text']]

In [14]:
data.iloc[3,:]

category                                           ENTERTAINMENT
merged_text    Jim Carrey Blasts 'Castrato' Adam Schiff And D...
Name: 3, dtype: object

In [15]:
data.category.value_counts()

POLITICS          32739
WELLNESS          17827
ENTERTAINMENT     16058
TRAVEL             9887
STYLE & BEAUTY     9649
PARENTING          8677
HEALTHY LIVING     6694
QUEER VOICES       6314
FOOD & DRINK       6226
BUSINESS           5937
COMEDY             5175
SPORTS             4884
BLACK VOICES       4528
HOME & LIVING      4195
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3651
WOMEN              3490
IMPACT             3459
DIVORCE            3426
CRIME              3405
MEDIA              2815
WEIRD NEWS         2670
GREEN              2622
WORLDPOST          2579
RELIGION           2556
STYLE              2254
SCIENCE            2178
WORLD NEWS         2177
TASTE              2096
TECH               2082
MONEY              1707
ARTS               1509
FIFTY              1401
GOOD NEWS          1398
ARTS & CULTURE     1339
ENVIRONMENT        1323
COLLEGE            1144
LATINO VOICES      1129
CULTURE & ARTS     1030
EDUCATION          1004
Name: category, 

In [0]:
def keep_values(df, col_name, values):
    return df.loc[df[col_name].isin(values), :].reset_index(drop=True)

In [0]:
df = keep_values(
    data, "category", data["category"].value_counts()[:15].index
)

In [18]:
df["category"].value_counts()

POLITICS          32739
WELLNESS          17827
ENTERTAINMENT     16058
TRAVEL             9887
STYLE & BEAUTY     9649
PARENTING          8677
HEALTHY LIVING     6694
QUEER VOICES       6314
FOOD & DRINK       6226
BUSINESS           5937
COMEDY             5175
SPORTS             4884
BLACK VOICES       4528
HOME & LIVING      4195
PARENTS            3955
Name: category, dtype: int64

In [0]:
def make_category_map(labels) -> Dict[str, int]:
    return {x: i for i, x in enumerate(sorted(set(labels)))}

In [0]:
def init_category_map(filename, labels):
    try:
        category_map = load_file(filename)
    except:
        category_map = make_category_map(labels)
        save_file(category_map, filename)
    return category_map

In [0]:
category_map = init_category_map(
    TEMP_DIR / "category_map.pickle", df["category"].values
)

In [22]:
category_map

{'BLACK VOICES': 0,
 'BUSINESS': 1,
 'COMEDY': 2,
 'ENTERTAINMENT': 3,
 'FOOD & DRINK': 4,
 'HEALTHY LIVING': 5,
 'HOME & LIVING': 6,
 'PARENTING': 7,
 'PARENTS': 8,
 'POLITICS': 9,
 'QUEER VOICES': 10,
 'SPORTS': 11,
 'STYLE & BEAUTY': 12,
 'TRAVEL': 13,
 'WELLNESS': 14}

In [0]:
n_categories = len(category_map)

In [24]:
n_categories

15

In [0]:
df["category"] = df["category"].map(category_map)

In [26]:
df.category.value_counts()

9     32739
14    17827
3     16058
13     9887
12     9649
7      8677
5      6694
10     6314
4      6226
1      5937
2      5175
11     4884
0      4528
6      4195
8      3955
Name: category, dtype: int64

In [0]:
all_data_bundle = DataBundle.from_dataframe(
    dataframe=df, x_col="merged_text", y_col="category"
)

In [0]:
train_data_bundle, validation_data_bundle, test_data_bundle = DataBundle.split(
    data_bundle=all_data_bundle, fracs=[0.1, 0.1, 0.1]
)

In [0]:
data_container = DataContainer(
    train_data_bundle, validation_data_bundle, test_data_bundle
)

In [30]:
data_container.train.value_counts()

{0: 413,
 1: 601,
 2: 498,
 3: 1618,
 4: 624,
 5: 660,
 6: 446,
 7: 837,
 8: 420,
 9: 3276,
 10: 640,
 11: 505,
 12: 960,
 13: 999,
 14: 1778}

In [0]:
class_weights = dict(
    enumerate(
        sk.utils.class_weight.compute_class_weight(
            "balanced", np.unique(data_container.train.y), data_container.train.y
        )
    )
)

In [0]:
data_container.train = DataBundle.from_unbalanced(data_container.train)

In [33]:
data_container.train.value_counts()

{0: 2891,
 1: 3005,
 2: 2988,
 3: 3236,
 4: 3120,
 5: 2640,
 6: 3122,
 7: 2511,
 8: 2940,
 9: 3276,
 10: 3200,
 11: 3030,
 12: 2880,
 13: 2997,
 14: 1778}

In [34]:
len(data_container.train), len(data_container.validation), len(data_container.test)

(43614, 14275, 14275)

In [0]:
def make_dataset(data_bundle):
    return tf.data.Dataset.from_tensor_slices((data_bundle.x, data_bundle.y))

In [0]:
data_container.train.dataset = make_dataset(data_container.train)
data_container.validation.dataset = make_dataset(data_container.validation)
data_container.test.dataset = make_dataset(data_container.test)

In [37]:
data_container.train.x[5]

"This MLK Quote Sums Up The Rise Of White Supremacy Post-Trump. King's words are eerily prophetic."

In [38]:
data_container.train.y[5]

0

In [0]:
def make_sentence_length_limiter(limit):
    def inner(x, y):
        return tf.strings.substr(x, 0, limit), y

    return inner

In [0]:
length_limiter = make_sentence_length_limiter(300)

In [0]:
train_dataset = (
    data_container.train.dataset.repeat()
    .shuffle(len(data_container.train))
    .batch(BATCH_SIZE)
    .map(length_limiter)
    .prefetch(AUTOTUNE)
)

validation_dataset = (
    data_container.validation.dataset.batch(BATCH_SIZE)
    .map(length_limiter)
    .prefetch(AUTOTUNE)
)

In [0]:
train_dataset_steps = math.ceil(len(data_container.train) / BATCH_SIZE)

## **Tf hub pretrained models**

In [0]:
def train_model(
    model,
    epochs,
    lrs=None,
    optimizers=None,
    patience=5,
    verbose=1,
    log_dir=str(TEMP_DIR / "logs"),
):
    if optimizers is None:
        optimizers = [keras.optimizers.Adam(lr) for lr in lrs]
    model.layers[0].trainable = False
    model.compile(
        loss=keras.losses.sparse_categorical_crossentropy,
        optimizer=optimizers[0],
        metrics=[
            keras.metrics.sparse_categorical_accuracy,
            sparse_top_2_categorical_accuracy,
        ],
    )
    model.fit(
        train_dataset,
        steps_per_epoch=train_dataset_steps,
        validation_data=validation_dataset,
        epochs=epochs[0],
        callbacks=[
            keras.callbacks.ReduceLROnPlateau(patience=1, factor=0.3),
            keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
        ],
        class_weight=class_weights,
        verbose=verbose,
    )
    model.layers[0].trainable = True
    model.compile(
        loss=keras.losses.sparse_categorical_crossentropy,
        optimizer=optimizers[1],
        metrics=[
            keras.metrics.sparse_categorical_accuracy,
            sparse_top_2_categorical_accuracy,
        ],
    )
    model.fit(
        train_dataset,
        steps_per_epoch=train_dataset_steps,
        validation_data=validation_dataset,
        epochs=epochs[1],
        callbacks=[
            keras.callbacks.ReduceLROnPlateau(patience=patience // 2, factor=0.3),
            keras.callbacks.EarlyStopping(patience=patience, restore_best_weights=True),
        ],
        class_weight=class_weights,
        verbose=verbose,
    )

In [0]:
def make_hub_model(url):
    return keras.Sequential(
        [
            hub.KerasLayer(url, dtype=tf.string, input_shape=[]),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(n_categories, activation=keras.activations.softmax),
        ]
    )

In [0]:
def run_models(urls):
    for url in urls:
        model = make_hub_model(url)
        model_name = f"{url.split('/')[4]}"
        print(f" {model_name} ".center(80, "="))
        shutil.rmtree(str(TEMP_DIR / model_name), ignore_errors=True)
        train_model(
            model=model,
            epochs=[2, 1],
            optimizers=[keras.optimizers.Adam(lr=1e-4), keras.optimizers.Adam(lr=3e-5)],
            patience=2,
            verbose=2,
            log_dir=str(TEMP_DIR / model_name),
        )
        model.save(f"{TEMP_DIR / model_name}.h5")
        save_keras_model(
            model,
            str(TEMP_DIR / model_name / "architecture"),
            str(TEMP_DIR / model_name / "weights"),
        )
        keras.backend.clear_session()
        del model
        keras.backend.clear_session()

In [0]:
model_urls = (
    "https://tfhub.dev/google/Wiki-words-250-with-normalization/2",
    "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2",
    "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
)

In [47]:
run_models(model_urls)

INFO:absl:Using /tmp/tfhub_modules to cache modules.


====================== Wiki-words-250-with-normalization =======================
Train for 5452 steps, validate for 1785 steps
Epoch 1/2
5452/5452 - 31s - loss: 2.5115 - sparse_categorical_accuracy: 0.1969 - sparse_top_2_categorical_accuracy: 0.3267 - val_loss: 2.4318 - val_sparse_categorical_accuracy: 0.3713 - val_sparse_top_2_categorical_accuracy: 0.5150
Epoch 2/2
5452/5452 - 29s - loss: 2.2903 - sparse_categorical_accuracy: 0.3057 - sparse_top_2_categorical_accuracy: 0.4666 - val_loss: 2.1992 - val_sparse_categorical_accuracy: 0.4062 - val_sparse_top_2_categorical_accuracy: 0.5480
Train for 5452 steps, validate for 1785 steps
5452/5452 - 371s - loss: 2.0962 - sparse_categorical_accuracy: 0.3901 - sparse_top_2_categorical_accuracy: 0.5634 - val_loss: 2.0560 - val_sparse_categorical_accuracy: 0.4528 - val_sparse_top_2_categorical_accuracy: 0.5960


INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2'.
INFO:absl:Downloaded https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2, Total size: 483.55MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2'.


====================== nnlm-en-dim128-with-normalization =======================
Train for 5452 steps, validate for 1785 steps
Epoch 1/2
5452/5452 - 30s - loss: 2.5833 - sparse_categorical_accuracy: 0.1699 - sparse_top_2_categorical_accuracy: 0.2898 - val_loss: 2.5124 - val_sparse_categorical_accuracy: 0.2937 - val_sparse_top_2_categorical_accuracy: 0.4556
Epoch 2/2
5452/5452 - 29s - loss: 2.3525 - sparse_categorical_accuracy: 0.3161 - sparse_top_2_categorical_accuracy: 0.4891 - val_loss: 2.2649 - val_sparse_categorical_accuracy: 0.4623 - val_sparse_top_2_categorical_accuracy: 0.6104
Train for 5452 steps, validate for 1785 steps
5452/5452 - 198s - loss: 2.1462 - sparse_categorical_accuracy: 0.4236 - sparse_top_2_categorical_accuracy: 0.6027 - val_loss: 2.1069 - val_sparse_categorical_accuracy: 0.4927 - val_sparse_top_2_categorical_accuracy: 0.6451


INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'.
INFO:absl:Downloaded https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1, Total size: 1.69MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'.


================================= tf2-preview ==================================
Train for 5452 steps, validate for 1785 steps
Epoch 1/2
5452/5452 - 29s - loss: 3.2819 - sparse_categorical_accuracy: 0.0954 - sparse_top_2_categorical_accuracy: 0.1777 - val_loss: 2.4293 - val_sparse_categorical_accuracy: 0.2301 - val_sparse_top_2_categorical_accuracy: 0.3509
Epoch 2/2
5452/5452 - 28s - loss: 2.6054 - sparse_categorical_accuracy: 0.1713 - sparse_top_2_categorical_accuracy: 0.2915 - val_loss: 2.2067 - val_sparse_categorical_accuracy: 0.3313 - val_sparse_top_2_categorical_accuracy: 0.4597
Train for 5452 steps, validate for 1785 steps
5452/5452 - 33s - loss: 2.4312 - sparse_categorical_accuracy: 0.2100 - sparse_top_2_categorical_accuracy: 0.3458 - val_loss: 2.1784 - val_sparse_categorical_accuracy: 0.3428 - val_sparse_top_2_categorical_accuracy: 0.4719


In [48]:
print(
    classification_report(
        data_container.validation.y, 
        load_keras_model(
            str(TEMP_DIR/model_urls[0].split('/')[4]/'architecture'),
            str(TEMP_DIR/model_urls[0].split('/')[4]/'weights'),
            custom_objects={'KerasLayer':hub.KerasLayer}
        ).predict(data_container.validation.x).argmax(axis=1)))


              precision    recall  f1-score   support

           0       0.24      0.13      0.17       486
           1       0.24      0.48      0.32       601
           2       0.32      0.22      0.26       521
           3       0.45      0.70      0.55      1589
           4       0.48      0.79      0.60       615
           5       0.22      0.25      0.23       706
           6       0.29      0.66      0.40       434
           7       0.22      0.41      0.29       880
           8       0.16      0.32      0.21       389
           9       0.81      0.67      0.73      3289
          10       0.47      0.31      0.37       623
          11       0.46      0.53      0.49       502
          12       0.88      0.35      0.50       963
          13       0.69      0.45      0.55       979
          14       0.00      0.00      0.00      1698

    accuracy                           0.45     14275
   macro avg       0.40      0.42      0.38     14275
weighted avg       0.47   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [49]:
print(
    classification_report(
        data_container.validation.y, 
        load_keras_model(
            str(TEMP_DIR/model_urls[1].split('/')[4]/'architecture'),
            str(TEMP_DIR/model_urls[1].split('/')[4]/'weights'),
            custom_objects={'KerasLayer':hub.KerasLayer}
        ).predict(data_container.validation.x).argmax(axis=1)))


              precision    recall  f1-score   support

           0       0.27      0.19      0.22       486
           1       0.30      0.54      0.38       601
           2       0.25      0.36      0.29       521
           3       0.51      0.67      0.58      1589
           4       0.48      0.87      0.62       615
           5       0.22      0.29      0.25       706
           6       0.39      0.77      0.51       434
           7       0.24      0.49      0.32       880
           8       0.20      0.34      0.25       389
           9       0.82      0.72      0.77      3289
          10       0.52      0.25      0.34       623
          11       0.71      0.46      0.56       502
          12       0.89      0.55      0.68       963
          13       0.85      0.45      0.59       979
          14       0.00      0.00      0.00      1698

    accuracy                           0.49     14275
   macro avg       0.44      0.46      0.42     14275
weighted avg       0.51   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [50]:
print(
    classification_report(
        data_container.validation.y, 
        load_keras_model(
            str(TEMP_DIR/model_urls[2].split('/')[4]/'architecture'),
            str(TEMP_DIR/model_urls[2].split('/')[4]/'weights'),
            custom_objects={'KerasLayer':hub.KerasLayer}
        ).predict(data_container.validation.x).argmax(axis=1)))


              precision    recall  f1-score   support

           0       0.08      0.11      0.09       486
           1       0.21      0.17      0.19       601
           2       0.15      0.18      0.16       521
           3       0.34      0.56      0.42      1589
           4       0.31      0.54      0.39       615
           5       0.14      0.23      0.18       706
           6       0.18      0.28      0.22       434
           7       0.19      0.46      0.27       880
           8       0.08      0.11      0.09       389
           9       0.78      0.64      0.70      3289
          10       0.17      0.10      0.13       623
          11       0.34      0.43      0.38       502
          12       0.42      0.03      0.06       963
          13       0.56      0.26      0.36       979
          14       0.33      0.00      0.00      1698

    accuracy                           0.34     14275
   macro avg       0.29      0.27      0.24     14275
weighted avg       0.40   

## **BERT model**

In [0]:
def make_bert_dataset(data_bundle, tokenizer, sequence_length):
    features = tf.data.Dataset.from_tensor_slices(
        [
            np.pad(
                tokenizer.encode(x, add_special_tokens=True),
                (0, sequence_length),
                "constant",
                constant_values=tokenizer.pad_token_id,
            )[:sequence_length]
            for x in data_bundle.x
        ]
    )
    labels = tf.data.Dataset.from_tensor_slices(data_bundle.y)
    dataset = tf.data.Dataset.zip((features, labels)).map(
        lambda x, y: (
            {
                "input_ids": x,
                "attention_mask": int(x != tokenizer.pad_token_id),
                "token_type_ids": int(x >= tokenizer.pad_token_id),
            },
            y,
        ),
        num_parallel_calls=AUTOTUNE,
    )
    return dataset

In [54]:
config = transformers.BertConfig.from_pretrained(
    "bert-base-cased", num_labels=n_categories
)

100%|██████████| 313/313 [00:00<00:00, 216708.01B/s]


In [0]:
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-cased")

In [57]:
train_bert_dataset = (
    make_bert_dataset(data_container.train, tokenizer, 128)
    .repeat()
    .shuffle(len(data_container.train))
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

In [0]:
validation_bert_dataset = (
    make_bert_dataset(data_container.validation, tokenizer, 128)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

In [59]:
model = transformers.TFBertForSequenceClassification.from_pretrained(
    "bert-base-cased", config=config
)

100%|██████████| 526681800/526681800 [00:10<00:00, 50255104.55B/s]


In [0]:
model.layers[0].trainable = False

In [0]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=1e-5),
    metrics=[keras.metrics.SparseCategoricalAccuracy("accuracy")],
)

In [0]:
def train_model_bert(
    model,
    epochs,
    train_dataset,
    train_dataset_steps,
    validation_dataset,
    lrs=None,
    optimizers=None,
    patience=5,
    verbose=1,
    log_dir=str(TEMP_DIR / "bert/logs"),
):
    model.layers[0].trainable = False
    if optimizers is None:
        optimizers = [keras.optimizers.Adam(lr) for lr in lrs]
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=optimizers[0],
        metrics=[
            keras.metrics.sparse_categorical_accuracy,
            sparse_top_2_categorical_accuracy,
        ],
    )
    model.fit(
        train_dataset,
        steps_per_epoch=train_dataset_steps,
        validation_data=validation_dataset,
        epochs=epochs[0],
        callbacks=[
            keras.callbacks.ReduceLROnPlateau(patience=1, factor=0.3),
            keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
        ],
        class_weight=class_weights,
        verbose=verbose,
    )
    model.layers[0].trainable = True
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=optimizers[1],
        metrics=[
            keras.metrics.sparse_categorical_accuracy,
            sparse_top_2_categorical_accuracy,
        ],
    )
    model.fit(
        train_dataset,
        steps_per_epoch=train_dataset_steps,
        validation_data=validation_dataset,
        epochs=epochs[1],
        callbacks=[
            keras.callbacks.ReduceLROnPlateau(patience=patience // 2, factor=0.3),
            keras.callbacks.EarlyStopping(patience=patience, restore_best_weights=True),
            keras.callbacks.TensorBoard(log_dir=log_dir),
        ],
        class_weight=class_weights,
        verbose=verbose,
    )

In [63]:
train_model_bert(
    model=model,
    epochs=[2, 1],
    train_dataset=train_bert_dataset,
    train_dataset_steps=train_dataset_steps // 5,
    validation_dataset=validation_bert_dataset,
    optimizers=[keras.optimizers.Adam(lr=1e-5), keras.optimizers.Adam(lr=1e-6)],
    patience=2,
    verbose=2,
)

Train for 1090 steps, validate for 1785 steps
Epoch 1/2
1090/1090 - 210s - loss: 2.6879 - sparse_categorical_accuracy: 0.0961 - sparse_top_2_categorical_accuracy: 0.1884 - val_loss: 2.8008 - val_sparse_categorical_accuracy: 0.0360 - val_sparse_top_2_categorical_accuracy: 0.0761
Epoch 2/2
1090/1090 - 186s - loss: 2.6103 - sparse_categorical_accuracy: 0.1351 - sparse_top_2_categorical_accuracy: 0.2579 - val_loss: 2.8139 - val_sparse_categorical_accuracy: 0.0965 - val_sparse_top_2_categorical_accuracy: 0.1626
Train for 1090 steps, validate for 1785 steps
1090/1090 - 320s - loss: 2.4725 - sparse_categorical_accuracy: 0.2236 - sparse_top_2_categorical_accuracy: 0.3678 - val_loss: 2.2502 - val_sparse_categorical_accuracy: 0.3282 - val_sparse_top_2_categorical_accuracy: 0.5212
